In [1]:
from bs4 import BeautifulSoup # Web Scrape
import pandas as pd
import numpy as np
import requests # Makes HTTP requests
# import time
import re # Regular expression

In [2]:
url = 'https://www.giants.com/team/stats/'
response = requests.get(url)
print("Response:", response.status_code,response.url) #200 means it went through

Response: 200 https://www.giants.com/team/stats/


In [3]:
giants_stats = BeautifulSoup(response.text,'html.parser')
giants_stats_body = giants_stats.body
nfl_div = giants_stats_body.find_all('div')
giants_stats_body_main_content = giants_stats_body.find(id="main-content")
# print(giants_stats_body_main_content.prettify())

In [35]:
offensive_passing_table = giants_stats_body_main_content.find("div", {"class": "nfl-o-teamstats"})
# print(offensive_passing_table.prettify())

In [37]:
headers = offensive_passing_table.find('thead')
# print(headers)

In [6]:
clean_headers = headers.find_all('th')
passing_headers = [item.get_text() for item in clean_headers]

print(passing_headers)

['Player', 'ATT', 'COMP', 'YDS', 'COMP%', 'YDS/ATT', 'TD', 'TD%', 'INT', 'INT%', 'LONG', 'SCK', 'SCK/LOST', 'RATE']


In [8]:

offensive_passing_body = offensive_passing_table.find('tbody')
# print(offensive_passing_body)


clean_offensive_passer_name = offensive_passing_body.find_all('span',{"class":"nfl-o-roster__player-name"})
# print(clean_offensive_passer_name)

passer_name = [item.get_text() for item in clean_offensive_passer_name] 
print(passer_name)



['Alex Tanney', 'Eli Manning', 'Daniel Jones']


In [40]:

clean_offensive_passer_stats = [item.get_text() for item in offensive_passing_body.find_all('td')]
# print(clean_offensive_passer_stats)

clean_offensive_passer_stats = [str(item).replace(' ','') for item in clean_offensive_passer_stats]
# print(clean_offensive_passer_stats)

pattern = re.compile(r'[\d{0,6}]*|[\d{0,6}\d{0,6}\.\d{0,6}]*')
matches = pattern.findall(str(clean_offensive_passer_stats))
# print(matches)

clean_matches = []

for match in matches:
    if match == '':
        continue
    else:
        clean_matches.append(match)

clean_matches = [str(item).replace(',',' ') for item in clean_matches]

# clean_matches = [str(item).split('\n') for item in clean_matches]
# print(clean_matches)




counter = 0
counter1 = 1
counter2 = -1
counter4 = 0 


for item in clean_matches:
    if '.' in item:
        rowData = clean_matches[counter2]+clean_matches[counter]
        clean_matches.pop(counter)
        clean_matches.pop(counter2)
        clean_matches.insert(counter,rowData)
    if item == ' ':
        clean_matches.pop(counter)
        
    counter = counter + 1
    counter1 = counter1 + 1
    counter2 = counter2 + 1

for item in clean_matches:
    if item == ' ':
        clean_matches.pop(counter4)
    counter4 = counter4 + 1

# print()
# print(clean_matches)


passing_data = clean_matches

print(passing_data)


['1', '1', '1', '100', '1', '0', '0', '0', '0', '1', '0', '0', '79.2', '147', '91', '1042', '61.9', '7.1', '6', '4.08', '5', '3.4', '55', '5', '44', '82.6', '459', '284', '3027', '61.87', '6.6', '24', '5.23', '12', '2.61', '75', '38', '295', '87.7']


In [41]:
B = np.zeros((len(passer_name),len(passing_headers)))
count = 0

B = B.astype('str')
B = B.tolist()

for i in range(0,len(passer_name)):
    for j in range(1,len(passing_headers)):
        B[i][j] = passing_data[count]
        B[i][0] = passer_name[i]
        count = count + 1 



print(B)

   

[['Alex Tanney', '1', '1', '1', '100', '1', '0', '0', '0', '0', '1', '0', '0', '79.2'], ['Eli Manning', '147', '91', '1042', '61.9', '7.1', '6', '4.08', '5', '3.4', '55', '5', '44', '82.6'], ['Daniel Jones', '459', '284', '3027', '61.87', '6.6', '24', '5.23', '12', '2.61', '75', '38', '295', '87.7']]


In [42]:
df = pd.DataFrame(B, columns = passing_headers)

df

,Player,ATT,COMP,YDS,COMP%,YDS/ATT,TD,TD%,INT,INT%,LONG,SCK,SCK/LOST,RATE
0,Alex Tanney,1,1,1,100,1,0,0,0,0,1,0,0,79.2
1,Eli Manning,147,91,1042,61.9,7.1,6,4.08,5,3.4,55,5,44,82.6
2,Daniel Jones,459,284,3027,61.87,6.6,24,5.23,12,2.61,75,38,295,87.7
